In [1]:
import numpy as np
import cv2

In [2]:
def on_click(event, x, y, flags, params):
    if event == cv2.EVENT_LBUTTONDOWN:
        params.append((x, y))

In [82]:
def get_correspondences(im1, im2):
    
    out = []
    
    for i, im in enumerate([im1, im2]):
        
        points = []
        cv2.namedWindow('image')
        cv2.setMouseCallback('image', on_click, points)

        while (True):
            cv2.imshow('image', im)
            if cv2.waitKey(20) == 27:
                break

        cv2.destroyAllWindows()
        
        out.append(list(points))
        
    return np.array(out[0]).T.astype(float), np.array(out[1]).T.astype(float)

In [90]:
def computeH(t1, t2):
    
    t1 = np.concatenate([t1, [[1] * n_points]], axis=0)
    
    L = []
    for i in range(n_points):
        L.append(np.concatenate([t1[:, i], [0] * 3, -t2[0, i] * t1[:, i]]))
        L.append(np.concatenate([[0] * 3, t1[:, i], -t2[1, i] * t1[:, i]]))
        
    L = np.array(L)
    A = np.matmul(L.T, L)
    values, vectors = np.linalg.eig(A)
    H = vectors[np.argmin(values)].reshape(3, 3)
    
    return H

In [93]:
im1 = cv2.imread('crop1.jpg')
im2 = cv2.imread('crop2.jpg')

In [94]:
t1, t2 = get_correspondences(im1, im2)

In [95]:
h, w, _ = im1.shape

In [96]:
n_points = t1.shape[1]

In [105]:
t1_scaled = t1 / np.array([[w/2], [h/2]])
t2_scaled = t2 / np.array([[w/2], [h/2]])

In [106]:
H = computeH(t1_scaled, t2_scaled)

In [107]:
H

array([[ 0.59797905, -0.00374291,  0.21283372],
       [ 0.1101865 ,  0.34382794,  0.37094798],
       [-0.16559739,  0.25941325,  0.48417973]])

In [110]:
t1_hom = np.concatenate([t1_scaled, [[1] * n_points]], axis=0)

In [113]:
p_prime = np.matmul(H, t1_hom[:, 0])

In [116]:
p_prime / p_prime[2]

array([24.41918917, 14.80557242,  1.        ])

In [117]:
t2

array([[116., 173., 238., 161.],
       [355., 229., 115., 114.]])

In [118]:
p_prime

array([216.29109928, 131.13922461,   8.85742347])